In [28]:

from pptx.util import Inches, Pt

In [15]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from dotenv import load_dotenv
load_dotenv()


class ChatAI:

    def __init__(self):
        self.model = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
        self.embeddings = OpenAIEmbeddings()
        self.search_tool = TavilySearchResults()
        instructions = """You are a power point presentation specialist."""
        base_prompt = hub.pull("langchain-ai/openai-functions-template")
        self.prompt = base_prompt.partial(instructions=instructions)
        self.tools = [self.search_tool]

    def ingest(self, file_path: str):
        docs = PyPDFLoader(file_path=file_path).load()
        chunks = self.text_splitter.split_documents(docs)
        chunks = filter_complex_metadata(chunks)
        vector_store = Chroma.from_documents(documents=chunks, embedding=self.embeddings)
        self.retriever = vector_store.as_retriever(search_type = "mmr", search_kwargs={
            "k": 6,
            "fetch_k": 20,
            "include_metadata": True
            },
        )
        self.retriever_tool = create_retriever_tool(
                self.retriever,
                "uploaded_files_search",
                "Search for information about uploaded files, for any question about uploaded files you should use this tool",
            )
        self.tools.append(self.retriever_tool)
        self.agent = create_openai_functions_agent(self.model, self.tools, self.prompt)



    def ask(self, questioin: str,num_slides: int):
        format_prompt ="""
        Example:
        {"slides": [
        {"title": "title for slide 1", "content": "Content for slide 1"},
        {"title": "title for slide 2", "content": "Content for slide 2"},
        {"title": "title for slide 3", "content": "Content for slide 3"},
        ]}
        """
        template = f"""
        question: {questioin}
        you should create a PowerPoint presentation to answer the above question. Make it {num_slides}  slides.
        Each slide should have a title and content.
        step 1: think and write an outline for each title.
        step 2: with the content, expand on each of the title of your outline using at least 3 to 5 sentences. 
        You can consider elaborating on the key ideas, offering supporting examples and explaining any details that you think would enhance the audience's understanding on the topic.
        step 3: Structure the output information as a JSON as follows.
        Your answer should only contain the JSON - no markdown formatting.
        """ + format_prompt

        agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=False)
        result = agent_executor.invoke({"input": template})
        return result["output"]


In [16]:
chai_ai = ChatAI()
chai_ai.ingest("test.pdf")

In [21]:
output = chai_ai.ask("What is the eight element of thinking using uploaded file", 4)
print(output)

{
  "slides": [
    {
      "title": "Introduction to the Eight Elements of Thinking",
      "content": "The eight elements of thinking are essential components that contribute to effective critical thinking. These elements provide a framework for analyzing and evaluating information critically. By understanding each element, individuals can enhance their problem-solving skills and make informed decisions."
    },
    {
      "title": "The First Element: Purpose",
      "content": "Purpose is the first element of thinking and involves identifying the goal or objective of the thinking process. It is crucial to clarify the purpose to ensure that the thinking is focused and directed towards achieving a specific outcome. Without a clear purpose, the thinking process may lack direction and coherence."
    },
    {
      "title": "The Second Element: Question at Issue",
      "content": "The second element, question at issue, emphasizes the importance of formulating relevant and meaningful q

In [22]:
import json
output = json.loads(output)

In [24]:
output['slides']

[{'title': 'Introduction to the Eight Elements of Thinking',
  'content': 'The eight elements of thinking are essential components that contribute to effective critical thinking. These elements provide a framework for analyzing and evaluating information critically. By understanding each element, individuals can enhance their problem-solving skills and make informed decisions.'},
 {'title': 'The First Element: Purpose',
  'content': 'Purpose is the first element of thinking and involves identifying the goal or objective of the thinking process. It is crucial to clarify the purpose to ensure that the thinking is focused and directed towards achieving a specific outcome. Without a clear purpose, the thinking process may lack direction and coherence.'},
 {'title': 'The Second Element: Question at Issue',
  'content': 'The second element, question at issue, emphasizes the importance of formulating relevant and meaningful questions. Effective thinking involves asking critical questions that

In [25]:
from pptx import Presentation


In [39]:
from openai import OpenAI
import os
import requests
from PIL import Image


# function for image generation
def image_generator(prompt):
    client = OpenAI()
    try:
        # Create an image by using the image generation API
        generation_response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size='1024x1024',
            n=1
        )
        image_url = generation_response.data[0].url # extract image URL from response
        image = requests.get(image_url).content  # download the image
        generated_image = BytesIO(image)
        return generated_image
    except client.error.InvalidRequestError as err:
        print(err)

In [51]:
def create_presentation(topic, slide_titles, slide_contents):
    prs = Presentation()
    title_side_layout = prs.slide_layouts[0]
    two_col_slide_layout = prs.slide_layouts[8]

    title_slide = prs.slides.add_slide(title_side_layout)
    title_slide.shapes.title.text = topic

    for slide_title, slide_content in zip(slide_titles, slide_contents):
        slide = prs.slides.add_slide(two_col_slide_layout)

        # add title
        slide.shapes.title.text = slide_title

        # add image 
        image = image_generator(slide_content)
        slide.shapes.placeholders[1].insert_picture(image)

        # Add text 
        slide.shapes.placeholders[2].text = slide_content
    

    prs.save(f"{topic}_presentation.pptx")

In [52]:
titles = [slide['title'] for slide in output['slides']]
contents = [slide['content'] for slide in output['slides']]

create_presentation("What is the eight element of thinking", titles, contents)